In [1]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())

                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [2]:
import pandas as pd

# CSV 또는 Parquet 파일을 불러온 뒤 df에 저장했다고 가정
# 예: df = pd.read_csv('your_file.csv') 또는 pd.read_parquet('your_file.parquet')

#  1. price가 0인 행 제거
df = df[df['price'] > 0]

#  2. brand 컬럼에서 결측치 제거
df = df.dropna(subset=['brand'])

# 결과 확인 (선택사항)
print(df.shape)
print(df['price'].min())
print(df['brand'].isnull().sum())

(11932733, 11)
0.06
0


In [3]:
import pandas as pd

# ✅ 1. 구매 데이터만 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()

# ✅ 2. 고객별 총 구매 횟수, 총 지출 금액, 평균 구매 금액 계산
customer_summary = purchase_df.groupby('user_key').agg(
    total_orders=('event_time', 'count'),
    total_spent=('price', 'sum'),
).reset_index()

customer_summary['avg_order_value'] = customer_summary['total_spent'] / customer_summary['total_orders']

# ✅ 3. 전체 고객 기준 평균 구매 빈도 추정
repeat_rate = customer_summary['total_orders'].mean()

# ✅ 4. 이탈률 추정 (간단 추정 방식)
churn_rate = 1 - (repeat_rate / customer_summary['total_orders'].max())

# ✅ 5. CLTV 계산
customer_summary['cltv'] = customer_summary['avg_order_value'] * repeat_rate / churn_rate

# ✅ 6. CLTV 기준 내림차순 정렬
customer_summary = customer_summary.sort_values(by='cltv', ascending=False)

# ✅ 7. 결과 일부 미리 보기
print("🎯 고객별 CLTV 상위 10명:")
print(customer_summary.head(10))

# ✅ 8. CSV로 저장
customer_summary.to_csv('customer_cltv.csv', index=False)
print("✅ 'customer_cltv.csv' 파일로 저장 완료!")


🎯 고객별 CLTV 상위 10명:
                                             user_key  total_orders  \
132404  61214809052c9eab9-f518-4844-94e0-019b6d07bb70             2   
77388   55841296720c79646-6996-42f3-a68f-5267c4fcdbcb             1   
103624  578383361163873b4-e873-4383-bd4e-319c76fd9530             1   
21765   43279676750f28793-8cef-4363-a57e-c0f1307b815f             1   
93096   56938061974d32f7c-bb4f-4cb8-84f4-924054cff462             1   
37227   4832405403f69cf61-2d03-4804-bf44-e538e7ba9e3e             1   
76585   55792755699bc375c-6068-18db-2e90-795ecf7c38fc             1   
98172   57359633946f38634-971d-4fb3-a24b-753a65d98c18             1   
32977   4724635128af772ba-0680-4443-b15c-e72d2c982d88             1   
87940   5657453796b73698f-db54-18ef-54a3-fb52ed3a4e5e             1   

        total_spent  avg_order_value         cltv  
132404       600.80           300.40  1641.161729  
77388        273.02           273.02  1491.577813  
103624       273.02           273.02  1491.

In [5]:
import pandas as pd

# ✅ 1. 구매 데이터만 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()

# ✅ 2. 고객별 총 구매 횟수, 총 지출 금액, 평균 구매 금액 계산
customer_summary = purchase_df.groupby('user_id').agg(
    total_orders=('event_time', 'count'),
    total_spent=('price', 'sum'),
).reset_index()

customer_summary['avg_order_value'] = customer_summary['total_spent'] / customer_summary['total_orders']

# ✅ 3. 전체 고객 기준 평균 구매 빈도 추정
repeat_rate = customer_summary['total_orders'].mean()

# ✅ 4. 이탈률 추정 (간단 추정 방식)
churn_rate = 1 - (repeat_rate / customer_summary['total_orders'].max())

# ✅ 5. CLTV 계산
customer_summary['cltv'] = customer_summary['avg_order_value'] * repeat_rate / churn_rate

# ✅ 6. CLTV 기준 내림차순 정렬
customer_summary = customer_summary.sort_values(by='cltv', ascending=False)

# ✅ 7. 결과 일부 미리 보기
print("🎯 고객별 CLTV 상위 10명:")
print(customer_summary.head(10))

# ✅ 8. CSV로 저장
customer_summary.to_csv('customer_cltv.csv', index=False)
print("✅ 'customer_cltv.csv' 파일로 저장 완료!")


🎯 고객별 CLTV 상위 10명:
         user_id  total_orders  total_spent  avg_order_value         cltv
86795  603739164             1       273.02           273.02  2085.976726
68764  578383361             1       273.02           273.02  2085.976726
14141  437699865             1       273.02           273.02  2085.976726
64364  573596339             1       273.02           273.02  2085.976726
48610  558412967             1       273.02           273.02  2085.976726
28246  509822747             1       273.02           273.02  2085.976726
13316  432796767             1       273.02           273.02  2085.976726
48025  557927556             2       467.46           233.73  1785.786170
90990  608809325             1       207.94           207.94  1588.740753
45470  555638848             1       207.94           207.94  1588.740753
✅ 'customer_cltv.csv' 파일로 저장 완료!
